In [1]:
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import geopandas as gpd

clubs_df = pd.read_feather('../cleaned_data/clubs.feather')
club_games_df = pd.read_feather('../cleaned_data/club_games.feather')
players_df = pd.read_feather('../cleaned_data/players.feather')
games_df = pd.read_feather('../cleaned_data/games.feather')
appearances_df = pd.read_feather('../cleaned_data/appearances.feather')